In [1]:
import compas_rrc as rrc
from compas.geometry import Point, Vector, Frame
import csv
import os

# Start ROS & ABB clients
ros = rrc.RosClient()
ros.run()

abb = rrc.AbbClient(ros, '/rob1')
print('Connected to ABB IRB 6700')


Connected to ABB IRB 6700


In [2]:
done = abb.send_and_wait(rrc.SetTool('fingergrab'))
frame = abb.send_and_wait(rrc.GetFrame())
print("Active TCP frame:", frame)


Active TCP frame: Frame(point=Point(x=-128.60736083984375, y=-1422.052734375, z=1375.447509765625), xaxis=Vector(x=-0.9992989818666221, y=0.03517704349830211, z=0.012810950435123971), yaxis=Vector(x=0.03544150167804624, y=0.9991503137234437, z=0.02103688534810866))


In [3]:
csv_filename = "pose_log.csv"
csv_path = os.path.join("D:\IAAC_24-25\MRAC_02\Collaborative_print\demo_rrc\logs", csv_filename)

with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["cycle", "direction", "x", "y", "z", "rx", "ry", "rz"])


In [5]:
def open_gripper():
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO8', 0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO6', 1))
    print(" → Gripper opened")

def close_gripper():
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO6', 0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO8', 1))
    print(" → Gripper closed")

frames = {
    "home":           Frame(Point(690.435653, -1306.975386, 1366.823567), Vector(0,1,0), Vector(1,0,0)),
    "safe_plane":     Frame(Point(-128.61, -1422.05,  1375.45), Vector(-0.999299,0.03518,0.012813), Vector(0.035444,0.99915,0.021035)),
    "approach_pick":  Frame(Point(180.61, -1394.42, 1077.14), Vector(0.999962,-0.007326,0.004801), Vector(-0.007193,-0.999608,-0.027055)),
    "pick":           Frame(Point(180.61, -1394.42, 977.14), Vector(0.999962,-0.007326,0.004801), Vector(-0.007192,-0.9996,-0.027335)),
    "approach_place": Frame(Point(819.34, -1824.85, 1109.19), Vector(0.999962,-0.007326,0.004801), Vector(-0.007193,-0.999608,-0.027055)),
    "place":          Frame(Point(819.34, -1824.85, 1007.19), Vector(0.999962,-0.007326,0.004801), Vector(-0.007192,-0.9996,-0.027335)),
}

speed_fly    = 500
speed_action = 20
zone         = rrc.Zone.FINE

# frames["place"].point.z += 25
# frames["approach_place"].point.z += 28

def do_move(step_name, frame, speed, motion):
    abb.send_and_wait(rrc.MoveToFrame(frame, speed, zone, motion))
    print(f" → Step '{step_name}' ({motion}) completed")


In [6]:
open_gripper()
abb.send_and_wait(rrc.WaitTime(2.0))
close_gripper()


 → Gripper opened
 → Gripper closed


In [7]:
def pick_and_place(pick_approach, pick, place_approach, place, cycle_num, direction):
    print(f"\n[CYCLE {cycle_num}] {direction}: {pick} → {place}")
    do_move(f"Approach {pick}",   frames[pick_approach],  speed_fly,    rrc.Motion.JOINT)
    open_gripper()
    do_move(f"Pick {pick}",       frames[pick],           speed_action, rrc.Motion.LINEAR)
    close_gripper()
    do_move(f"Retract {pick}",    frames[pick_approach],  speed_action, rrc.Motion.LINEAR)
    do_move(f"Approach {place}",  frames[place_approach], speed_fly,    rrc.Motion.JOINT)
    do_move(f"Place {place}",     frames[place],          speed_action, rrc.Motion.LINEAR)

    log_pose_to_csv(cycle_num, direction)

    abb.send_and_wait(rrc.WaitTime(1.0))
    open_gripper()
    abb.send_and_wait(rrc.WaitTime(1.0))
    do_move(f"Retract {place}",   frames[place_approach], speed_action, rrc.Motion.LINEAR)


In [8]:
def log_pose_to_csv(cycle, direction):
    pose = abb.send_and_wait(rrc.GetFrame())
    pos = pose.point
    xaxis = pose.xaxis
    with open(csv_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([
            cycle, direction,
            round(pos.x, 3), round(pos.y, 3), round(pos.z, 3),
            round(xaxis.x, 3), round(xaxis.y, 3), round(xaxis.z, 3)
        ])
    print(f" → Pose logged for cycle {cycle} ({direction})")


In [9]:
do_move("Start at safe_plane", frames["safe_plane"], speed_fly, rrc.Motion.JOINT)

for i in range(1, 21):  # 20 cycles
    if i % 2 == 1:
        pick_and_place("approach_pick", "pick", "approach_place", "place", i, "Forward")
    else:
        pick_and_place("approach_place", "place", "approach_pick", "pick", i, "Return")

do_move("Return to safe", frames["safe_plane"], speed_fly, rrc.Motion.JOINT)
print("All 20 cycles completed. Shutting down.")
ros.close()


 → Step 'Start at safe_plane' (J) completed

[CYCLE 1] Forward: pick → place
 → Step 'Approach pick' (J) completed
 → Gripper opened
 → Step 'Pick pick' (L) completed
 → Gripper closed
 → Step 'Retract pick' (L) completed
 → Step 'Approach place' (J) completed
 → Step 'Place place' (L) completed
 → Pose logged for cycle 1 (Forward)
 → Gripper opened
 → Step 'Retract place' (L) completed

[CYCLE 2] Return: place → pick
 → Step 'Approach place' (J) completed
 → Gripper opened
 → Step 'Pick place' (L) completed
 → Gripper closed
 → Step 'Retract place' (L) completed
 → Step 'Approach pick' (J) completed
 → Step 'Place pick' (L) completed
 → Pose logged for cycle 2 (Return)
 → Gripper opened
 → Step 'Retract pick' (L) completed

[CYCLE 3] Forward: pick → place
 → Step 'Approach pick' (J) completed
 → Gripper opened
 → Step 'Pick pick' (L) completed
 → Gripper closed
 → Step 'Retract pick' (L) completed
 → Step 'Approach place' (J) completed
 → Step 'Place place' (L) completed
 → Pose logge